In [13]:
import cobra
from cobra.io import load_json_model, read_sbml_model, write_sbml_model, save_json_model
import pandas as pd
import os

#Simplify reading/writing files
cwd=os.path.realpath(os.path.join(os.path.dirname(os.getcwd()),"..",".."))

# Quicker way to read in models
import pickle
def read_model(fileName, obj='none'):
    
    fileType = fileName.split('.')[-1]
    
    if fileType == 'sbml' or fileType == 'xml':
        model = cobra.io.read_sbml_model(fileName)
    elif fileType == 'json':
        model = cobra.io.load_json_model(fileName)
    elif fileType == 'yaml':
        model = cobra.io.load_yaml_model(fileName)
    elif fileType == 'mat':
        model = cobra.io.load_matlab_model(fileName)
    elif fileType == 'pkl':
        model = pickle.load(open(fileName, 'rb'))
    else:
        raise TypeError('Unrecognized file extension')
    
    if obj != 'none': model.objective = obj
    for rxn in model.boundary: rxn.bounds = (-1000., 1000.)
        
    return model

In [3]:
model=read_model(cwd+'/Gc_GENRE_2022/Generate_Gc_Model/FinalModels/AE004969.1.json')

In [4]:
model.metabolites.M_thr_L_c_c

Metabolite identifier,M_thr_L_c_c
Name,M_thr_L_c
Memory address,0x0247660dff10
Formula,
Compartment,Cytosol
In 6 reaction(s),"THRS, THRD_L, THRt2pp, PROTS, THRabcpp, THRt1rpp"


In [5]:
#Remove M_ and _c from each metabolite id
for metabolite in model.metabolites:
    if 'M_' in metabolite.id:
        metabolite.id= metabolite.id[2:-2]

In [6]:
model.metabolites.thr_L_c

Metabolite identifier,thr_L_c
Name,M_thr_L_c
Memory address,0x0247660dff10
Formula,
Compartment,Cytosol
In 6 reaction(s),"THRS, THRD_L, THRt2pp, PROTS, THRabcpp, THRt1rpp"


In [8]:
#load metabolite annotation table
metabolites= list()
for mets in model.metabolites:
    metabolites.append(mets.id)
metabolites

#Use Mendum 2011 metabolite formulas
annotation_table = pd.read_csv (cwd+'/Gc_GENRE_2022/Generate_Gc_Model/Mendum_2011_annotations/metabolite_annotations.csv', usecols= ['Abbreviated name', 'Full name', 'Formula' ],index_col=0)
annotation_table = annotation_table.fillna('')
annotation_table                       


,Full name,Formula
Abbreviated name,,
10fthf_c,10-Formyltetrahydrofolate,C20H21N7O7
12dgr120_c,1-2-Diacyl-sn-glycerol-didodecanoyl-n-C120,C27H52O5
12dgr120_p,1-2-Diacyl-sn-glycerol-didodecanoyl-n-C120,C27H52O5
12dgr140_c,1-2-Diacyl-sn-glycerol-ditetradecanoyl-n-C140,C31H60O5
12dgr140_p,1-2-Diacyl-sn-glycerol-ditetradecanoyl-n-C140,C31H60O5
...,...,...
NaN,,
NaN,,
NaN,,


In [9]:
#Add gene annotations to model from annotation table
for x in model.metabolites:
    metid=x.id
    if metid in annotation_table.index:
        full_name_id=annotation_table['Full name'].loc[metid]
        model.metabolites.get_by_id(metid).name=str(full_name_id)
        formula=annotation_table['Formula'].loc[metid]
        model.metabolites.get_by_id(metid).formula= str(formula)      
    else:
        pass

In [10]:
model.metabolites.thr_L_c.formula

'C4H9NO3'

In [11]:
model

Name,AE004969.1
Memory address,0x02475ed68820
Number of metabolites,1297
Number of reactions,1484
Number of groups,0
Objective expression,0
Compartments,Cytosol


In [14]:
save_json_model(model,cwd+'/Gc_GENRE_2022/Models/AE004969.1.json')